In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import xgi
from governance import *

In [ ]:
def create_decision_matrix(m):
    D = np.zeros((m, m))
    for i in range(m):
        for j in range(m):
            D[i, j] = random.choice([-1, 0, 1])
    return D

def create_opinions(n, m):
    D = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            D[i, j] = random.uniform(-1, 1)
    return D

In [ ]:
opinions = create_opinions(100, 20)
decision_matrix = create_decision_matrix(20)
group_size = 3
group_overlap = 1

In [ ]:
decision_process(opinions, decision_matrix, select_decision, select_group, make_decision, update_opinions, group_size, group_overlap)